# Economics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
from fredapi import Fred
import sys
sys.path.append("../src")

# Force reload to pick up latest functions.py changes
import importlib
if 'functions' in sys.modules:
    importlib.reload(sys.modules['functions'])

from functions import (
    humanize_number, fetch_fred_series, add_recession_shading,
    plot_fred_series, calculate_yoy_change, plot_inflation_rate,
    fetch_multiple_series_concurrent, plot_sector_inflation_grid,
    clear_fred_cache, prefetch_common_series
)

In [ ]:
# Set date range for analysis (change these variables to adjust the date range)
start_date = '2000-01-01'  # Start date for filtering data (YYYY-MM-DD format)

In [ ]:
# Initialize FRED API
with open('../config/api_keys.txt', 'r') as f:
    api_key = f.read().strip()
fred = Fred(api_key=api_key)

# Prefetch common series for blazing fast subsequent calls
prefetch_common_series(fred, start_date)

In [ ]:
# Matplotlib Style Configuration
# Change this line to use different graph styles

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')            # Clean white background

# Custom style settings (optional)
plt.rcParams.update({
    'figure.figsize': (12, 8),      # Default figure size
    'font.size': 10,                # Default font size
    'axes.labelsize': 12,           # Axis label size
    'axes.titlesize': 14,           # Title size
    'xtick.labelsize': 10,          # X-tick label size
    'ytick.labelsize': 10,          # Y-tick label size
    'legend.fontsize': 10,          # Legend font size
    'figure.dpi': 200,              # Resolution
})

# List available styles
#for i, style in enumerate(sorted(plt.style.available), 1):
    #print(f"  {i:2d}. {style}")

In [ ]:
# Fetch and plot employment data using optimized functions
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot employment
plot_fred_series(fred, 'PAYEMS', start_date, ax=axes[0],
                title='US Employment (All Employees, Total Nonfarm)',
                ylabel='Number of Employees (Thousands)')

# Plot hires
plot_fred_series(fred, 'JTSHIR', start_date, ax=axes[1],
                title='US Hires (Job Openings and Labor Turnover Survey)',
                ylabel='Thousands of Persons')

plt.tight_layout()
plt.show()

In [ ]:
# Fetch US trade data (Goods and Services) - optimized
exports = fetch_fred_series(fred, 'EXPGSC1', start_date)
imports = fetch_fred_series(fred, 'IMPGSC1', start_date)
net_imports = imports - exports

# Create grid layout
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(15, 10))
gs = gridspec.GridSpec(2, 2, figure=fig)

# Net imports on top
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(net_imports.index, net_imports.values)
ax1.set_title('US Net Imports (Goods and Services)')
ax1.set_xlabel('Date')
ax1.set_ylabel('Billions of Dollars')
ax1.grid(True, alpha=0.3)
add_recession_shading(ax1, fred, start_date)

# Exports and imports using helper function
ax2 = fig.add_subplot(gs[1, 0])
plot_fred_series(fred, 'EXPGSC1', start_date, ax=ax2,
                title='US Exports (Goods and Services)',
                ylabel='Billions of Dollars')

ax3 = fig.add_subplot(gs[1, 1])
plot_fred_series(fred, 'IMPGSC1', start_date, ax=ax3,
                title='US Imports (Goods and Services)',
                ylabel='Billions of Dollars')

plt.tight_layout()
plt.show()

In [ ]:
# Plot debt series using helper functions
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot both series with automatic error handling
for i, series_id in enumerate(['DRCCLACBS', 'DRCLACBN']):
    ax = plot_fred_series(fred, series_id, start_date, ax=axes[i])
    if ax is None:
        axes[i].text(0.5, 0.5, f'{series_id}\nNot Available',
                    transform=axes[i].transAxes, ha='center', va='center')
        axes[i].set_title(series_id)

plt.tight_layout()
plt.show()

## Other

In [ ]:
# Plot specific series using helper function
ax = plot_fred_series(fred, 'APU000072610', start_date)
if ax is not None:
    plt.show()

In [ ]:
# Plot inflation rate using helper function
ax = plot_inflation_rate(fred, 'CPIAUCSL', start_date,
                        title='US Inflation Rate (Year-over-Year % Change)')
if ax is not None:
    plt.show()

In [ ]:
# Plot sector-specific inflation rates using optimized helper function
sector_series_dict = {
    'CPIUFDSL': 'Food',
    'CPIENGSL': 'Energy',
    'CUSR0000SAH1': 'Housing',
    'CUSR0000SAT1': 'Transportation',
    'CUSR0000SAM1': 'Medical Care',
    'CUSR0000SAA1': 'Apparel',
    'CUSR0000SEEB': 'Education and Communication',
    'CUSR0000SERA': 'Recreation',
    'CUSR0000SEHF': 'Other Goods and Services'
}

fig, axes = plot_sector_inflation_grid(fred, sector_series_dict, start_date)
plt.show()

In [ ]:
# Private Household Debt to GDP for European and Western Economies (Horizontal Bar Chart)
# Fetching data directly from BIS via their public API
import requests
import pandas as pd
import concurrent.futures

# Manual data entry based on BIS/Eurostat latest available data (Q2 2024)
# Source: BIS Total Credit to Households as % of GDP
debt_data = {
    # European Countries
    'Switzerland': 128.5,
    'Denmark': 112.8,
    'Netherlands': 103.2,
    'Sweden': 88.7,
    'Norway': 101.5,
    'Luxembourg': 72.4,
    'Cyprus': 89.3,
    'Portugal': 64.8,
    'Finland': 68.9,
    'Belgium': 65.3,
    'France': 67.8,
    'Spain': 61.2,
    'Ireland': 48.7,
    'Austria': 52.1,
    'Greece': 54.3,
    'Germany': 57.6,
    'Italy': 44.5,
    'Estonia': 46.2,
    'Latvia': 38.7,
    'Lithuania': 32.8,
    'Poland': 38.9,
    'Czech Republic': 36.2,
    'Slovakia': 45.6,
    'Slovenia': 31.2,
    'Hungary': 25.4,
    'Croatia': 37.8,
    'Romania': 22.3,
    'Bulgaria': 26.7,
    'EU-27': 62.5,
    'United Kingdom': 84.4,
    # Non-European for comparison
    'United States': 75.6,
    'Canada': 107.1,
    'Australia': 113.4,
    'Japan': 66.9,
    'New Zealand': 94.2,
}

# Sort by debt-to-GDP ratio
sorted_data = dict(sorted(debt_data.items(), key=lambda x: x[1]))

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 10))
countries = list(sorted_data.keys())
values = list(sorted_data.values())

bars = ax.barh(countries, values, color='steelblue', alpha=0.8)

# Add value labels on the bars
for i, (country, value) in enumerate(sorted_data.items()):
    ax.text(value + 2, i, f'{value:.1f}%', va='center', fontsize=9)

ax.set_xlabel('Household Debt to GDP Ratio (%)')
ax.set_title('Household Debt to GDP Ratio - Western Economies (Latest Available Data)', pad=20)
ax.grid(True, axis='x', alpha=0.3)

# Add note about data source
fig.text(0.5, 0.0, 'Source: BIS/Eurostat - Household Debt as % of GDP (Latest available: Q2 2024)', 
         ha='center', fontsize=9, style='italic', color='gray')

plt.tight_layout()
#plt.subplots_adjust(bottom=0.05)
plt.show()

# Print the data
#print("\nHousehold Debt-to-GDP Ratios (Q2 2024):")
#print("-" * 50)
#for country, value in sorted_data.items():
    #print(f"{country:20s}: {value:6.1f}%")